In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise02AI import Exercise02AI, display_path

# Aufgabe 03: Minimax (Simplified Evaluation Function)

Diese Klasse implementiert die Simplified Evaluation Function zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise02AI`) aufgerufen und durch die nun implementierte Simplified Evaluation Function verbessert.

Hierzu werden zu Beginn die in der [Simplified Evaluation Function](https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables) beschriebenen `Piece-Square Tables` definiert. Diese weisen jeder der 64 möglichen Positionen auf dem Schachfeld für alle der Schachfiguren (Bauer, Springer, Läufer, Turm, Dame und König) eine numerische Bewertung in Form einer Ganzzahl zu. Diese kann sowohl positiv (gute Position für die Figur) oder negativ (schlechte Position für die Figur) sein und wird als Liste dargestellt. Der König verfügt über zwei verschiedene Listen, eine für das Mittelspiel, eine für das Endspiel.

Anhand der Piece-Square Table der Bauern `PST_PAWN`, die im Folgenden definiert ist, wird ersichtlich, dass Bauern auf den Feldern `f2`, `g2` und `h2` (symmetrischer Weise natürlich auch auf `a2`, `b2` und `c2`) Boni (positive Bewertung) erhalten, während sie auf den Felder `f3` und `g3` Abzüge (negative Bewertungen) erhalten.

Das Dictionary `PIECE_TO_PST` weist jedem Figurentyp die entsprechende umgedrehte Piece-Square Table zu, welche aufgrund der einfacheren Lesbarkeit in umgekehrter Reihenfolge initialisiert werden.

In [ ]:
import chess

class Exercise03AI(Exercise02AI):
    """Chooses middle game moves using minimax algorithm and piece square tables"""
    def __init__(self, player_name: str, search_depth: int = 3) -> None:
        super().__init__(player_name, search_depth)
        # Create pice square table mapping (needs to revert tables) 
        self.PIECE_TO_PST = {
            chess.PAWN:   self.PST_PAWN[::-1],
            chess.KNIGHT: self.PST_KNIGHT[::-1],
            chess.BISHOP: self.PST_BISHOP[::-1],
            chess.ROOK:   self.PST_ROOK[::-1],
            chess.QUEEN:  self.PST_QUEEN[::-1],
            chess.KING:   self.PST_KING[::-1]
        }
        self.PIECE_TO_PST_KING_ENDGAME = {
            chess.PAWN:   self.PST_PAWN[::-1],
            chess.KNIGHT: self.PST_KNIGHT[::-1],
            chess.BISHOP: self.PST_BISHOP[::-1],
            chess.ROOK:   self.PST_ROOK[::-1],
            chess.QUEEN:  self.PST_QUEEN[::-1],
            chess.KING:   self.PST_KING_ENDGAME[::-1]
        }

    # Define field values as class constants
    PST_PAWN = [
        0,  0,  0,  0,  0,  0,  0,  0,
       50, 50, 50, 50, 50, 50, 50, 50,
       10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]

    PST_KNIGHT = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]

    PST_BISHOP = [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20
    ]

    PST_ROOK = [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ]

    PST_QUEEN = [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
         -5,  0,  5,  5,  5,  5,  0, -5,
          0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ]

    PST_KING = [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
         20, 20,  0,  0,  0,  0, 20, 20,
         20, 30, 10,  0,  0, 10, 30, 20
    ]

    PST_KING_ENDGAME = [
        -50,-40,-30,-20,-20,-30,-40,-50,
        -30,-20,-10,  0,  0,-10,-20,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-30,  0,  0,  0,  0,-30,-30,
        -50,-30,-30,-30,-30,-30,-30,-50
    ]

## Prüfung der Endspielkonditionen

Um bei der Simplified Evaluation Function die richtige Königstabelle auszuwählen ist es notwendig zu prüfen, ob sich das aktuelle Spiel im End- oder im Mittelspiel befindet. Das Endspiel wird dabei definiert als ein Spiel bei dem entweder:

1. beide Seiten keine Dame mehr besitzen oder
2. jede Seite die eine Dame hat, keinen Turm und maximal einen Läufer oder Springer besitzt.

**Hinweis**: Diese Definition der Endspielkonditionen bezieht sich nur auf die Simplified Evaluation Function. Allgemein ist das Endspiel (implementiert durch die Syzygy-Endspielbibliothek) erreicht, sobald 5 oder weniger Figuren auf dem Spielbrett vorhanden sind.

Die Funktion `is_king_endgame` implementiert die oben genannten Bedingungen und prüft, ob sie für ein gegebenes Board erfüllt sind.

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    def is_king_endgame(self, board: chess.Board) -> bool:
        """Checks if the king endgame tables should be used."""
        return all((
            (
                not board.pieces(chess.QUEEN, color)
                or len(board.pieces(chess.BISHOP, color)) + len(board.pieces(chess.KNIGHT, color)) < 2 
                and not board.pieces(chess.ROOK, color)
            )
            for color in chess.COLORS
        ))

## Evaluierungsfunktion

Die Funktion `calculate_pst_value` nimmt als Argument ein Board, einen Figurentyp und eine Farbe. Anhand der gegebenen Farbe wird auf dem Board nach den Figuren des gegebenen Typs gefiltert, deren Position auf dem Board bestimmt (für schwarze Figuren wird das Board gespiegelt) und mithilfe der Piece-Square Tables eine Bewertung dieser vollzogen. Diese Bewertungen werden, falls schwarz als Farbe gegeben ist, negiert. Die Bewertungen werden im letzten Schritt aufsummiert und zurückgegeben.

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    def calculate_pst_value(self, piece: chess.PieceType, color: chess.Color, positions: chess.SquareSet, is_king_endgame: bool) -> int:
        """Calculates the piece-square table values for a given piece, color and and optional position"""
        tables = self.PIECE_TO_PST if not is_king_endgame else self.PIECE_TO_PST_KING_ENDGAME
        if color == chess.WHITE:
            return sum([tables[piece][i] for i in positions])
        else:
            return -sum([(tables[piece][chess.square_mirror(i)]) for i in positions])

Die Funktion `evaluate` berechnet den Wert eines gesamten Boards, welches als Argument übergeben wird. Als Grundlage werden die Materialwerte der einzelnen Figuren in Centipawns benötigt, weshalb die bereits in der Elternklasse (`Exercise02AI`) gesetzten Werte wie folgt überschrieben werden:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 100  |
| Springer (knight) | 320  |
| Läufer (bishop) | 330  |
| Turm (rook) | 500  |
| Dame (queen) | 900  |
| König (king) | 20000 |

Die folgenden Funktionen der Elternklasse wurden überschrieben:

- `full_evaluate`
- `incremental_evaluate`

Die Funktion `full_evaluate` berechnet den Wert eines Boards von Grund auf neu und geht dabei wie folgt vor:

1. Es wird für jede Figur und für jede Farbe mithilfe der Funktion `calculate_pst_value` die Bewertung der Figur in Centipawns ermittelt.
2. Es wird die Summe über alle Bewertungen gebildet. Je größer die Summe, desto besser ist Weiß auf dem Feld positioniert, umgekehrt ist ein negatives Ergebnis ein Indikator dafür, dass Schwarz besser auf dem Board positioniert ist.
3. Die Summe aller Bewertungen wird auf den Rückgabewert der `full_evaluate` Funktion der Elternklasse addiert und zurückgegeben.

Der Rückgabewert der Funktion beschreibt also die Kombination des reinen Materialwertes der Figuren beider Spieler und der Positionsbewertung der `Piece-Square Tables`.

Die Funktion `incremental_evaluate` berechnet die Differenz der Bewertung, die durch den letzten Zug entstanden ist. Hierbei werden die folgenden Fälle beachtet:

- Umwandlung eines Bauern in eine höherwertige Figur
- Veränderung der Positionsbewertung der gezogenen Figur
- Rochade
- Schlagen einer Figur
- Übergang in das Endspiel

Wird ein Bauer in eine höherwertige Figur umgewandelt, so wird die alte Positionsbewertung des gezogenen Bauern von der Bewertung der umgewandelten Figur subtrahiert. Das Ergebnis wird in der Variable `change` gespeichert.
War der letzte Zug keine Umwandlung, so wird die alte Bewertung der gezogenen Figur von ihrer neuen Bewertung abgezogen. Das Ergebnis wird ebenfalls in `change` gespeichert.

Handelt es sich bei dem letzten Zug um eine lange Rochade, so ändert sich die Positionsbewertung des Turms. Hierzu wird der Materialwert eines Turms auf `change` addiert, falls Weiß am Zug ist, oder von `change` subtrahiert, falls Schwarz am Zug ist. Bei einer kurzen Rochade entsteht keine Veränderung der Positionsbewertung durch den bewegten Turm.

Falls während des letzten Zuges eine Figur geschlagen wurde, wird die alte Bewertung der geschlagenen Figur errechnet und von `change` subtrahiert. Hierbei wird eine separate Abfrage verwendet, um zu erkennen, ob es sich bei dem letzten Zug um einen en passent Zug handelt.

Zuletzt wird überprüft, ob ein Übergang in das Endspiel stattgefunden hat, da der König über eine separate `Piece-Square Table` für das Endspiel verfügt. Hat ein solcher Übergang stattgefunden, so werden die neuen und alten Positionsbewertungen beider Könige berechnet. Die neuen Bewertungen werden auf `change` addiert, während die alten Bewertungen subtrahiert werden.

Zurückgegeben wird schlussendlich die Summe aus `change` und dem Rückgabewert der Funktion `incremental_evaluate` der Elternklasse.

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    MATERIAL_VALUES = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 20000
    }


    def full_evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board."""
        is_king_endgame = self.is_king_endgame(board)
        # Get if pieces standing well or pieces standing badly by calculating with the Piece-Square Tables
        rating = sum(
            self.calculate_pst_value(piece_type, color, board.pieces(piece_type, color), is_king_endgame)
            for piece_type in chess.PIECE_TYPES
            for color in chess.COLORS
        )
        return rating + super().full_evaluate(board)


    def incremental_evaluate(self, board: chess.Board, last_move: chess.Move, is_king_endgame: bool) -> int:
        """Returns an incrementally calculated evaluation of the given board."""
        change = 0

        if promotion_piece_type := last_move.promotion:
            # Calculate change caused by piece promotion
            old_piece_pst = self.calculate_pst_value(chess.PAWN, board.turn, [last_move.from_square], is_king_endgame)
            new_piece_pst = self.calculate_pst_value(promotion_piece_type, board.turn, [last_move.to_square], is_king_endgame)
            change += new_piece_pst - old_piece_pst
            # Check if endgame status has changed
            if self.is_king_endgame(board) != is_king_endgame:
                old_rating_king_white = self.calculate_pst_value(chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE), not is_king_endgame)
                old_rating_king_black = self.calculate_pst_value(chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK), not is_king_endgame)
                new_rating_king_white = self.calculate_pst_value(chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE), is_king_endgame)
                new_rating_king_black = self.calculate_pst_value(chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK), is_king_endgame)
                change += (new_rating_king_white + new_rating_king_black) - (old_rating_king_white + old_rating_king_black)
        else:
            # Calculate change caused by piece move
            piece_type = board.piece_type_at(last_move.from_square)
            old_pst_value = self.calculate_pst_value(piece_type, board.turn, [last_move.from_square], is_king_endgame)
            new_pst_value = self.calculate_pst_value(piece_type, board.turn, [last_move.to_square], is_king_endgame)
            change += new_pst_value - old_pst_value

        
        if captured_piece_type := board.piece_type_at(last_move.to_square):
            # Calculate additional change caused by piece capture
            captured_piece_pst = self.calculate_pst_value(captured_piece_type, not board.turn, [last_move.to_square], is_king_endgame)
            change -= captured_piece_pst
            # Check if endgame status has changed
            if self.is_king_endgame(board) != is_king_endgame:
                old_rating_king_white = self.calculate_pst_value(chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE), not is_king_endgame)
                old_rating_king_black = self.calculate_pst_value(chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK), not is_king_endgame)
                new_rating_king_white = self.calculate_pst_value(chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE), is_king_endgame)
                new_rating_king_black = self.calculate_pst_value(chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK), is_king_endgame)
                change += (new_rating_king_white + new_rating_king_black) - (old_rating_king_white + old_rating_king_black)
        elif board.is_en_passant(last_move):
            # Calculate additional change caused by en passant pawn capture
            pawn_double_move = board.peek()
            captured_pawn_pst = self.calculate_pst_value(chess.PAWN, not board.turn, [pawn_double_move.to_square], is_king_endgame)
            change -= captured_pawn_pst
        elif board.is_queenside_castling(last_move):
            # Add changed rook value caused by long castling (short castling does not change rook value and can be ignored)
            change += 5 * (1 if board.turn else -1)

        return change + super().incremental_evaluate(board, last_move)


    def evaluate(self, board: chess.Board, last_evaluation: int = None) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        # Do full evaluation if no last evaluation given
        if last_evaluation is None:
            return self.full_evaluate(board)
        # Incremental evaluation
        is_king_endgame = self.is_king_endgame(board)
        last_move = board.pop()
        evaluation = last_evaluation + self.incremental_evaluate(board, last_move, is_king_endgame)
        board.push(last_move)
        return evaluation

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
player = Exercise03AI("Testplayer", 3)

In [ ]:
last_eval = player.evaluate(board)
last_eval

In [ ]:
board.push(chess.Move.from_uci("d4c3"))
board

In [ ]:
print(player.evaluate(board, last_eval))
print(player.evaluate(board))

In [ ]:
from time import time
def time_full_evals(num):
    board = chess.Board()
    player = Exercise03AI("Testplayer", 3)
    last_eval = player.evaluate(board)
    start = time()
    for _ in range(num):
        move = list(board.legal_moves)[0]
        board.push(move)
        last_eval = player.evaluate(board)
    end = time()
    return end - start

In [ ]:
def time_inc_evals(num):
    board = chess.Board()
    player = Exercise03AI("Testplayer", 3)
    last_eval = player.evaluate(board)
    start = time()
    for _ in range(num):
        move = list(board.legal_moves)[0]
        board.push(move)
        last_eval = player.evaluate(board, last_eval)
    end = time()
    return end - start

In [ ]:
time_full_evals(10000)

In [ ]:
time_inc_evals(10000)

In [ ]:
import chess.pgn
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
# Test evaluate
board = chess.Board("4k3/5p2/8/6P1/8/8/8/4K3 b - - 0 1")
board.push(chess.Move.from_uci('f7f5')) # two square pawn move
player = Exercise03AI("Testplayer", 3)
player.evaluate(board)
board.push(chess.Move.from_uci('g5f6')) # en passant capture
player.evaluate(board)

In [ ]:
# Check castling move
board = chess.Board("4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1")
queenside_castling_move = list(board.legal_moves)[-1]
print(board.is_queenside_castling(queenside_castling_move))
player = Exercise03AI("Testplayer", 3)
player.get_cache_evaluation(board)
board.push(queenside_castling_move)
player.evaluate(board)

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("5rk1/1b3p2/3b3r/3p4/3p2P1/5Q1N/3q1PB1/R3R1K1 b - - 8 33")
player = Exercise03AI("Testplayer", DEPTH)
# print(player.minimax(board, DEPTH + 1))
move = player.get_next_middle_game_move(board)
print(move)
#display_path(board, move, DEPTH)